# Lecture 1: Bayesian Inference

by Martin Robinson

```
Oxford Research Software Engineering group (http://www.cs.ox.ac.uk/projects/RSE)
Department of Computer Science
University of Oxford
Pints: https://github.com/pints-team/pints
```


# Who am I?

- PhD in Mathematical Science at Monash University
- Now a:
    - Senior Research Software Engineer in the [Oxford RSE group](http://www.cs.ox.ac.uk/projects/RSE)
    - Co-director of EPSRC & MRC [Sustainable Approaches to Biomedical Science Centre for Doctoral Training: Responsible and Reproducible Research](https://sabsr3.web.ox.ac.uk), or SABS:R3 
- Research Interests include:
    - numerical modelling and simulation
    - particle-based methods
    - Bayesian inference
    - developing robust and reliable software for research
        - see [Aboria](https://github.com/aboria/Aboria), Chaste, PINTS, Smoldyn, SPH-DEM, PyBaMM and Trase
    

# Course Structure

- Lecture 1: Bayesian Inference
- Lecture 2: Maximum Likelihood Estimation
- Lecture 3: MCMC sampling
- Lecture 4: Hierarchical models

# Lecture Structure

- What is Bayesian Inference?
- Bayes Theorem
- Introdcution to Pints

# Acknowledgements

# What is Bayesian Statistics?

- A statistical philosophy, or a way of thinking about probabilities

**Frequentist:** $\;P(A)\;$ describes the limiting frequency of an event $A$. there is a fixed value of $\;P(A)\;$ that must be calculated e.g. proportion of heads from a fair coin toss will approach 0.5 after a large number of trials

**Bayesian:** $\;P(A)\;$ is a measure of centainty, quantification of investigators belief that $\;A\;$ is true a fixed value of $\;P(A)\;$ is not neccessary, nor desirable. Pior information must be used to augment sample data


# What is Bayesian Inference?



An method of inference, that uses Bayes' theorem to update prior beliefs after obtaining new data $y$

- *Likelihood function:* the probability of obtaining the data $y$, given a set of parameters $\theta$
- *Prior probability distribution:* encodes your uncertainty in the parameters before the data $y$ has been obtained

```
                     Bayes' Theorem
likelihood + prior -------------------> posterior
```

- *Posterior distribution:* updated probability distribution of $\theta$, given the new data $y$

# Tangible benefits of Bayesian inference

- Straightforward application to scientific modelling and experimentatal data analysis
- Simple and intuitive model building (unlike frequentist statistics there is no need to remember lots of specific formulae).
- Exhaustive and creative model testing.
- Straightforward interpretation of results.
